In [1]:
!pip install Cmake

  Using cached cmake-3.27.5-py2.py3-none-win_amd64.whl (34.6 MB)


In [2]:
!pip install scikit-surprise

  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-win_amd64.whl size=1093717 sha256=e2f5cdeaf757a127dd54e589ad36cd7fdad926ebabce4d7804c08365ccc2acd4
  Stored in directory: c:\users\sreya\appdata\local\pip\cache\wheels\c6\3a\46\9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [3]:
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [4]:
ratings = pd.read_csv('ratings.csv')

In [5]:
ratings.head()

,userId,movieId,rating,tstamp
0,206,4803,4.0,2003-04-07 13:52:01
1,5073,72731,4.0,2020-02-19 16:07:53
2,4739,91653,4.0,2020-12-28 15:35:58
3,535,3005,3.0,2008-12-26 05:38:11
4,465,4776,3.0,2008-08-13 20:22:36


In [6]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3908657 entries, 0 to 3908656
Data columns (total 4 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   movieId  int64  
 2   rating   float64
 3   tstamp   object 
dtypes: float64(1), int64(2), object(1)
memory usage: 119.3+ MB


In [7]:
ratings['rating'].value_counts()

4.0    900243
3.5    747522
3.0    693019
4.5    405031
5.0    341782
2.5    310870
2.0    241348
1.5     95228
1.0     95044
0.5     78570
Name: rating, dtype: int64

In [8]:
min_rating = 0.5
max_rating = 5.0

# Create a Surprise Reader object to specify the rating scale
reader = Reader(rating_scale=(min_rating, max_rating))  # Specify the actual rating scale used in your data

# Load the Pandas DataFrame into a Surprise Dataset
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=.2)


In [9]:
algo = SVD()

In [10]:
algo.fit(trainset)

In [11]:
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.7436


0.7435690337462179

In [18]:
def predict_rating(user_id, movie_id):
    prediction = algo.predict(user_id, movie_id)
    return prediction.est

In [19]:
predict_rating(206, 4803)

3.2047122582033762

In [13]:
def recommend_movies(user_id):
    # Get a list of all the movies the user has not rated
    all_movies = ratings['movieId'].unique()
    user_movies = ratings[ratings['userId'] == user_id]['movieId'].unique()
    new_movies = list(set(all_movies) - set(user_movies))
    
    # Predict the ratings for the new movies
    predictions = [algo.predict(user_id, movie_id) for movie_id in new_movies]
    
    # Sort the predictions by estimated rating
    predictions.sort(key=lambda x: x.est, reverse=True)
    
    # Get the top 10 recommendations
    top_recommendations = [prediction.iid for prediction in predictions[:10]]
    
    return top_recommendations

In [14]:
movie_ids=recommend_movies(206)

In [15]:
df=ratings = pd.read_csv('movies.csv')

In [16]:
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
filtered_df = df[df['movieId'].isin(movie_ids)]

# Iterate over the filtered DataFrame and print the title and genres for each movie
for index, row in filtered_df.iterrows():
    print(f"Movie ID: {row['movieId']}")
    print(f"Title: {row['title']}")
    print(f"Genres: {row['genres']}")
    print('-' * 30)

Movie ID: 1361
Title: Paradise Lost: The Child Murders at Robin Hood Hills (1996)
Genres: Documentary
------------------------------
Movie ID: 6896
Title: Shoah (1985)
Genres: Documentary|War
------------------------------
Movie ID: 26073
Title: Human Condition III, The (Ningen no joken III) (1961)
Genres: Drama|War
------------------------------
Movie ID: 134849
Title: Duck Amuck (1953)
Genres: Animation|Children|Comedy
------------------------------
Movie ID: 171011
Title: Planet Earth II (2016)
Genres: Documentary
------------------------------
Movie ID: 182723
Title: Cosmos: A Spacetime Odissey
Genres: (no genres listed)
------------------------------
